In [297]:
import pandas as pd
import matplotlib.pyplot as plt
import re
import mmap
import os

In [298]:
uname = !whoami
uname = uname[0]

In [299]:
# Names of the temporary directories (sl for slash)
mgtemp = '/Users/'+ uname +'/Work/Expts/EMBARC/Data/PRT/Massachusetts/MGTemp'
mgtempsl = mgtemp + '/'
umtemp = '/Users/'+ uname +'/Work/Expts/EMBARC/Data/PRT/Michigan/UMTemp'
umtempsl = umtemp + '/'
cutemp = '/Users/'+ uname +'/Work/Expts/EMBARC/Data/PRT/New_York/CUTemp'
cutempsl = cutemp + '/'
txtemp = '/Users/'+ uname +'/Work/Expts/EMBARC/Data/PRT/Texas/TXTemp'
txtempsl = txtemp + '/'

# Makes all of the temporary directories
!mkdir {mgtemp}
!mkdir {umtemp}
!mkdir {cutemp}
!mkdir {txtemp}

In [300]:
mg = '/Users/'+ uname +'/Work/Expts/EMBARC/Data/PRT/Massachusetts/embarc_CU_MG0*'
um = '/Users/'+ uname +'/Work/Expts/EMBARC/Data/PRT/Michigan/embarc_CU_UM0*'
cu = '/Users/'+ uname +'/Work/Expts/EMBARC/Data/PRT/New_York/embarc_CU_CU0*'
tx = '/Users/'+ uname +'/Work/Expts/EMBARC/Data/PRT/Texas/embarc_CU_TX0*'

# the data files in the folder
mgfolders = !ls -d {mg}
umfolders = !ls -d {um}
cufolders = !ls -d {cu}
txfolders = !ls -d {tx}

In [301]:
# takes in the list of folders in the state, the temp file, and what to split the name by
def grabdata(statefolders, temp, split_by):
    for folder in statefolders:
        sub = folder.split('_')[split_by]
        new_name = sub + '_out.txt'
        new_dir = temp + new_name
        !cp {folder}/done/sigdet_output*out {new_dir}

In [302]:
# grabs the data for all state folders
# output lists people who don't have the data folder
grabdata(mgfolders, mgtempsl, 2)
grabdata(umfolders, umtempsl, 2)
grabdata(cufolders, cutempsl, 3)
grabdata(txfolders, txtempsl, 2)


cp: /Users/mlm2/Work/Expts/EMBARC/Data/PRT/Massachusetts/embarc_CU_MG0026MGBP1R1_flankerprt_20121208/done/sigdet_output*out: No such file or directory
cp: /Users/mlm2/Work/Expts/EMBARC/Data/PRT/Michigan/embarc_CU_UM0012UMBP1R1_flankerprt_20120628/done/sigdet_output*out: No such file or directory
cp: /Users/mlm2/Work/Expts/EMBARC/Data/PRT/Michigan/embarc_CU_UM0094UMBP1R1_flankerprt_20141113/done/sigdet_output*out: No such file or directory
cp: /Users/mlm2/Work/Expts/EMBARC/Data/PRT/New_York/embarc_CU_CU0012CUBP1R1_flankerprt_20120515/done/sigdet_output*out: No such file or directory
cp: /Users/mlm2/Work/Expts/EMBARC/Data/PRT/New_York/embarc_CU_CU0012CUBP1R1_flankerprt_20130308/done/sigdet_output*out: No such file or directory
cp: /Users/mlm2/Work/Expts/EMBARC/Data/PRT/New_York/embarc_CU_CU0017UMBP1R1_flankerprt_20120608/done/sigdet_output*out: No such file or directory
cp: /Users/mlm2/Work/Expts/EMBARC/Data/PRT/New_York/embarc_CU_CU0018CUBP1R1_flankerprt_20120515/done/sigdet_output*out:

In [303]:
# for easier referencing 
cusplit = '/Users/'+ uname +'/Work/Expts/EMBARC/Data/PRT/New_York/CUSplit/'
mgsplit = '/Users/'+ uname +'/Work/Expts/EMBARC/Data/PRT/Massachusetts/MGSplit/'
umsplit = '/Users/'+ uname +'/Work/Expts/EMBARC/Data/PRT/Michigan/UMSplit/'
txsplit = '/Users/'+ uname +'/Work/Expts/EMBARC/Data/PRT/Texas/TXSplit/'

cufinal = '/Users/'+ uname +'/Work/Expts/EMBARC/Data/PRT/New_York/CUFinal/'
mgfinal = '/Users/'+ uname +'/Work/Expts/EMBARC/Data/PRT/Massachusetts/MGFinal/'
umfinal = '/Users/'+ uname +'/Work/Expts/EMBARC/Data/PRT/Michigan/UMFinal/'
txfinal = '/Users/'+ uname +'/Work/Expts/EMBARC/Data/PRT/Texas/TXFinal/'

In [304]:
# Makes the directories where the split data files will go
!mkdir {cusplit}
!mkdir {mgsplit}
!mkdir {umsplit}
!mkdir {txsplit}

# Makes the directories where the final data files will go
!mkdir {cufinal}
!mkdir {mgfinal}
!mkdir {umfinal}
!mkdir {txfinal}

In [305]:
# finds the given key in this file
def findmatch(key, split_by, tempsl, filename):
    rx=r''+key+'(.*)'
    f = open(tempsl + filename, 'r', encoding='windows-1252')
    for eachline in f:
        string=re.match(rx, eachline)
        if string:
            return (string.group().split(' ')[split_by])
    else:
        return ''


In [306]:
def main(temp, tempsl, split, final, loc):
    for filename in os.listdir(temp):
        
        #searching for matches in the original file
        richkey = findmatch('Rich', 2, tempsl, filename)
        leankey = findmatch('Lean key: ', 2, tempsl, filename)
        subid = findmatch('Subject ID: ', 2, tempsl, filename)
        date = findmatch('Date: ', 1, tempsl, filename)
        bias = findmatch('Bias: ', 1, tempsl, filename)
        richstim = findmatch('Rich stimulus: ', 2,  tempsl, filename)
        leanstim = findmatch('Lean stimulus: ', 2, tempsl, filename)
        newfile = open(tempsl + filename, 'r', encoding='windows-1252').read().split('\n\n\n')
        sub = filename.split('_')[0]
        names = [ 'X' + sub +'_info.txt', sub +'.txt']
        
        #splits the the file into two, saves the data one in split folder
        for num, file in enumerate(newfile):
            open(split + names[1],'w').write(file)
        
        #open as df, add the info from the regular expression portion to the df, save as csv
        try:
            df = pd.read_csv(split + sub +'.txt', '\t')
            df["ID"]= loc + subid
            df['Site']= loc
            df['Date']= date
            df['Bias']= bias 
            df['RichStim']= richstim
            df['LeanStim']= leanstim
            df['LeanKey']= leankey
            df['RichKey']= richkey
            df.to_csv(final + sub + '.csv')
        except:
            pass
        

In [307]:
# pass all state files to the main function
main(umtemp, umtempsl, umsplit, umfinal, 'UM')
main(mgtemp, mgtempsl, mgsplit, mgfinal, 'MG')
main(cutemp, cutempsl, cusplit, cufinal, 'CU')
main(txtemp, txtempsl, txsplit, txfinal, 'TX')